In [2]:
from dotenv import load_dotenv
import os
import pandas as pd
import alpaca_trade_api as tradeapi
import requests
import json
import time
load_dotenv("alphavantage_api_key.env")

True

In [3]:
key = os.getenv("alphavantage_api_key")
print (type(key))

<class 'str'>


In [4]:
content = requests.get(url= "https://www.alphavantage.co/query?",  params = {"function":"BALANCE_SHEET","symbol":"AAL", "apikey":key})

In [5]:
content.json().keys()

dict_keys(['symbol', 'annualReports', 'quarterlyReports'])

In [7]:
airline_list = ["ALK", "UAL", "LUV", "DAL", "AL"]
tech_list = ["AAPL", "GOOGL", "AMZN", "MSFT", "FB"]

for symbol in airline_list:
    content = requests.get(url= "https://www.alphavantage.co/query?",  params = {"function":"BALANCE_SHEET", "symbol": symbol, "alphavantage_api_key": key})
    data = content.json()
    data_test=data['quarterlyReports']
    df = pd.DataFrame.from_dict(data_test)
    df.set_index('fiscalDateEnding', inplace=True)
    
    file_name = f'Data/Airlines/{symbol}_quarterly_reports.csv'
    df.to_csv(file_name)

KeyError: 'quarterlyReports'

In [9]:
symbol_list = ["AAL", "UAL", "LUV", "DAL", "AL"]
tech_list = ["AAPL", "GOOGL", "AMZN", "MSFT", "FB"]

for symbol in symbol_list:
    stock_price = requests.get('https://www.alphavantage.co/query?', params={'function':'TIME_SERIES_MONTHLY_ADJUSTED','symbol':symbol,
                                                                              'apikey':'Alpha_vantage_Api_key'})
    stock_price = stock_price.json()
    try:
        stock_price = stock_price['Monthly Adjusted Time Series']
    except KeyError:
        print('KeyError. Trying again in 60 seconds...')  
        time.sleep(60)
        stock_price = requests.get('https://www.alphavantage.co/query?', params={'function':'TIME_SERIES_MONTHLY_ADJUSTED','symbol':symbol,
                                                                              'apikey':'Alpha_vantage_Api_key'})
        stock_price = stock_price['Monthly Adjusted Time Series']

    data = {}
    date_list = []
    close_list = []
    for date in stock_price:
        date_list.append(date)
        close_list.append(stock_price[date]['4. close'])
    
    data = {'date': date_list, 'close': close_list}
    price_df = pd.DataFrame.from_dict(data)
    
    price_df.set_index('date', inplace=True)
    file_name = f'Data/Prices/{symbol}.csv'
    price_df.to_csv(file_name)

In [ ]:
symbol_list = ["AAPL"]
#symbol_list = ["AAL", "UAL", "LUV", "DAL", "AL"]
for symbol in symbol_list:
    stock_price = requests.get('https://www.alphavantage.co/query?', params={'function':'TIME_SERIES_MONTHLY_ADJUSTED','symbol':symbol,
                                                                              'apikey':'Alpha_vantage_Api_key'})
    stock_price = stock_price.json()
    try:
        stock_price = stock_price['Monthly Adjusted Time Series']
    except KeyError:
        print('KeyError. Trying again in 60 seconds...')  
        time.sleep(60)
        stock_price = requests.get('https://www.alphavantage.co/query?', params={'function':'TIME_SERIES_MONTHLY_ADJUSTED','symbol':symbol,
                                                                              'apikey':'Alpha_vantage_Api_key'})
        stock_price = stock_price['Monthly Adjusted Time Series']
    data = {}
    date_list = []
    close_list = []
    for date in stock_price:
        data[date] = stock_price[date]['4. close']
        date_list.append(date)
        close_list.append(stock_price[date]['4. close'])
    data = {'date': date_list, 'close': close_list}
    price_df = pd.DataFrame.from_dict(data)
    price_df.columns = ['date', 'close']
    price_df.set_index('date', inplace=True)
    price_df_ascending = price_df.sort_values(by="date", ascending=True)
    price_df_average = price_df_ascending.rolling(3).mean() 
    price_df_average = price_df_ascending.iloc[::3, :].truncate(before = "2019-03-29")
    final_month = price_df_ascending.iloc[-1] # we wanna append final_month to price_df_average
    price_df_average.append(final_month) 
    file_name = f'Data/Prices/{symbol}.csv'
    price_df.to_csv(file_name)
    print(f'{symbol}:')
    print(price_df_average)
    print('---')
    print(final_month)

KeyError. Trying again in 60 seconds...
